In [6]:
import pandas as pd
import numpy as np
import pickle
import gymnasium as gym
from gymnasium.spaces import Box, Discrete
import statistics
from utils import methods
from utils.rl_environments import GammaIntervalsEnv1

with open(methods.file_path('prof_data/data1_intervals.pkl', 'data'), 'rb') as f:
    intervals = pickle.load(f)

df = pd.read_csv(methods.file_path('prof_data/data1.csv', 'data'))

df['intervals'] = intervals
display(df.head())



,total,travel_time,h,c,intervals
0,43.0,3.535281,5.320843,1.787906,"[0.1306593104459917, 0.08214531495921755, 0.02..."
1,48.0,41.459338,0.120220,1.670595,"[4.276706517433805, 3.2544721073860834, 3.7400..."
2,31.0,114.792988,0.146738,6.669483,"[16.079087401217105, 11.608519241181346, 14.23..."
3,52.0,976.197274,0.003427,0.570308,"[28.206915811705233, 6.758341759637238, 24.521..."
4,39.0,41.909987,0.105977,3.407713,"[0.9262261308152662, 1.9254123526889755, 3.236..."


In [ ]:
from stable_baselines3 import DQN
model = DQN.load("dqn")

env = GammaIntervalsEnv1()
rewards = {}
print(env._get_info())
for i in range(len(df)):
    row = df.iloc[i]
    state, _ = env.reset(row=row)
    done = False
    total_reward = 0
    while not done:
        action, _ = model.predict(state)  
        state, reward, done, _, info = env.step(action)
        total_reward += reward
    rewards[i] = total_reward
    # print(f"Episode {i} reward: {total_reward} | h = {info['state']['h']}, c = {info['state']['c']}")
    print(f"Episode {i} reward: {total_reward} | cur_time = {info['state']['cur_time']}")


df['rewards'] = rewards

df.head()

{'hidden': {'alpha': -1, 'beta': -1, 'interval': -1, 'cum_sum_intervals': -1}, 'state': {'n': -1, 'N': -1, 'h': -1, 'c': -1, 'travel_time': -1, 'cur_time': -1, 'mean_n': -1, 'std_n': -1, 'alpha_hat': -1, 'beta_hat': -1, 'u_star_hat': -1, 'last_update': -1}}
Episode 0 reward: -1.7879058522669198 | h = 5.320843026119598, c = 1.7879058522669198
Episode 1 reward: -14.384762760681054 | h = 0.1202198590573862, c = 1.6705948753367512
Episode 2 reward: -41.76448053842139 | h = 0.1467379719105914, c = 6.66948298227321
Episode 3 reward: -0.27765786160272293 | h = 0.0034273522843131, c = 0.5703075042056449
Episode 4 reward: -6.323440413370132 | h = 0.1059768957857552, c = 3.40771285701576
Episode 5 reward: -100.68757550934495 | h = 0.0057529219480287, c = 2.5466310300259014
Episode 6 reward: -16.33518834971634 | h = 3.5939018771398312, c = 16.33518834971634
Episode 7 reward: -1.9833158405455957 | h = 0.1523792926763642, c = 4.193467919560364
Episode 8 reward: -9.706491950481718 | h = 0.1414820101

,total,travel_time,h,c,intervals,rewards
0,43.0,3.535281,5.320843,1.787906,"[0.1306593104459917, 0.08214531495921755, 0.02...",-1.787906
1,48.0,41.459338,0.120220,1.670595,"[4.276706517433805, 3.2544721073860834, 3.7400...",-14.384763
2,31.0,114.792988,0.146738,6.669483,"[16.079087401217105, 11.608519241181346, 14.23...",-41.764481
3,52.0,976.197274,0.003427,0.570308,"[28.206915811705233, 6.758341759637238, 24.521...",-0.277658
4,39.0,41.909987,0.105977,3.407713,"[0.9262261308152662, 1.9254123526889755, 3.236...",-6.323440


In [8]:
import plotly.graph_objects as go

# Calculate means
means = df[['rewards', 'h', 'c']].mean()

# Plot
fig = go.Figure(data=[go.Bar(x=means.index, y=means.values)])
fig.update_layout(title="Mean Values of Rewards, H, and C", yaxis_title="Mean Value")
fig.show()

# Calculate medians
medians = df[['rewards', 'h', 'c']].median()

# Plot
fig = go.Figure(data=[go.Bar(x=medians.index, y=medians.values)])
fig.update_layout(title="Median Values of Rewards, H, and C", yaxis_title="Median Value")
fig.show()







